In [ ]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os
import pandas as pd
import cvxpy as cp
import numpy as np


from portfolio_utils import load_datas, f_returns_on_df, f_mu_on_df, f_sigma_on_df
from level2.functions import optimize

# Charger les données
df = load_datas()

In [ ]:
df[df.index.year == 2015]

In [ ]:
# Calcul des rendements logarithmiques
returns = f_returns_on_df(df)

# Calcul des paramètres pour l'optimisation
mu = f_mu_on_df(returns)  # Annualisation (252 jours boursiers)
Sigma = f_sigma_on_df(returns)  # Annualisation de la matrice de covariance
num_assets = len(mu)


In [ ]:
c = 0.001  # Coût proportionnel de transaction
w0 = np.array(num_assets * [0])  # Portefeuille
#w0[0] = 0.5  # 50% dans le premier actif
#w0[1] = 0.3  # 30% dans le deuxième actif
w0[2] = 1  # 20% dans le troisième actif

K = 10  # Nombre d'actifs à sélectionner

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Récupérer les portefeuilles réparés valides
frontier_yields, frontier_volatility, frontier_cost, frontier_weights = optimize(mu, Sigma, w0, K, delta_tol=0.01, population_size=300, generations=1000, c=c)

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=15, azim=25)

ax.scatter(frontier_yields, frontier_volatility, frontier_cost)

ax.set_xlabel('Rendement')
ax.set_ylabel('Volatilité')
ax.set_zlabel('Coût')

plt.title("Front de Pareto NSGA-II (3 objectifs)")
plt.show()

In [ ]:
# Max rendement
max_return_index = np.argmax(frontier_yields)
w_max = frontier_weights[max_return_index]
print(f"Portefeuille avec rendement maximal (exact {K} actifs) :")
print(f"Rendement : {frontier_yields[max_return_index]:.4f}")
print(f"Volatilité : {frontier_volatility[max_return_index]:.4f}")
print(f"Coût : {frontier_cost[max_return_index]:.4f}")
for i, weight in enumerate(w_max):
    if weight > 1e-4:
        print(f"  {df.columns[i]} : {weight:.4f}")

# Min volatilité
min_vol_index = np.argmin(frontier_volatility)
w_min = frontier_weights[min_vol_index]
print(f"\nPortefeuille avec risque minimal (exact {K} actifs) :")
print(f"Rendement : {frontier_yields[min_vol_index]:.4f}")
print(f"Volatilité : {frontier_volatility[min_vol_index]:.4f}")
print(f"Coût : {frontier_cost[min_vol_index]:.4f}")
for i, weight in enumerate(w_min):
    if weight > 1e-4:
        print(f"  {df.columns[i]} : {weight:.4f}")